In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os, pickle, shutil, random, PIL
from PIL import Image

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader,random_split,Dataset, ConcatDataset ,SubsetRandomSampler 
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.transforms import v2

import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

from training_utils import *
from cam_loss_training_utils import *
from classification_models import *
from focal_loss_with_smoothing import *

from torchcam.methods import SmoothGradCAMpp, LayerCAM, GradCAM,CAM
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image, resize
from torch.nn.functional import softmax, interpolate
from copy import deepcopy
from sklearn.metrics import accuracy_score

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

DFNAME = 'MB_34_Layer_1_L2_freezeb5'
device = torch.device('cuda:0')
criterion1 = FocalLossWithSmoothing(num_classes = 2, gamma=2, lb_smooth = 0.1)
criterion2 =  nn.MSELoss()
# criterion1 = nn.CrossEntropyLoss()

modelname = 'MB_34_Layer_1_L2_freezeb5'
n_epochs = 50
batch_size = 8

In [ ]:
train_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/imgs/'
test_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/'
val_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/val/imgs/'

cbis_test_dir = '/home/aminul/CVL/cbis_ddsm_test/imgs/'

# train_dir = '/home/aminul/CVL/cs791_project/mass_vs_non_mass/train_original/imgs/'
# test_dir = '/home/aminul/CVL/cs791_project/mass_vs_non_mass/test_original/imgs/'
# val_dir = '/home/aminul/CVL/cs791_project/mass_vs_non_mass/val_original/imgs/'

In [ ]:
size = (1024,768)
train_set_whole = ImageFolder(train_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.RandomVerticalFlip(0.5),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(15),
    # v2.RandomAutocontrast(0.5),
    # v2.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.3, hue=0.1),
    # v2.RandomChannelPermutation(),
    v2.RandomAdjustSharpness(2,0.5),
    # v2.RandomAutocontrast(0.5),
    v2.Resize(size),
    # transforms.GaussianBlur(kernel_size=3),
    # transforms.RandomRotation(30),
    v2.ToTensor(),
]))

val_set = ImageFolder(val_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.Resize(size),
    v2.ToTensor(),
]))

test_set = ImageFolder(test_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.Resize(size),
    v2.ToTensor(),
]))

cbis_test_set = ImageFolder(cbis_test_dir,transform = transforms.Compose([
    # v2.Grayscale(1),
    v2.Resize(size),
    v2.ToTensor(),
]))

In [ ]:
train_loader = DataLoader(train_set_whole, batch_size=batch_size, shuffle=True, num_workers = 4)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers = 4)
test_loader_2 = DataLoader(test_set, batch_size=1, shuffle=False)

test_loader_cbis = DataLoader(cbis_test_set, batch_size=1, shuffle=False)

NameError: name 'val_set' is not defined

In [ ]:
model = custom_rn(2).to(device)

c = 0
for name, param in model.named_parameters():
    if name[3] == '5': ### 9 if we use ef5
        print(1)
        break
    param.requires_grad = False
    c +=  1

In [ ]:
optim = torch.optim.Adam(model.parameters(),lr=0.0002, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=3)

history = {'train_loss': [], 'valid_loss': [],'train_acc':[],'valid_acc':[],'cam_loss':[]}

for epoch in range(n_epochs):
    
    # train_loss, train_acc = train_model(model,train_loader,criterion1,optim,None,device,epoch)
    # valid_loss, valid_acc = test_model(model,valid_loader, criterion1, optim,modelname,device,epoch)
    
    train_loss, cam_loss, train_acc = cam_loss_train_model(model,train_loader,criterion1,criterion2,optim,None,device,epoch)  
    valid_loss, valid_acc = cam_loss_test_model(model,valid_loader,criterion1,optim,modelname,device,epoch)
    
    scheduler.step(valid_loss)
    
    # test(model,test_loader,criterion,optim,filename,modelname,device,epoch)
            
    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)
    
    history['cam_loss'].append(cam_loss)

with open('./storage/' + DFNAME + '.pkl', 'wb') as f:
    pickle.dump(history, f) 


In [ ]:
new_model =custom_rn(2).to(device)

checkpoint = torch.load('./checkpoint/'+modelname+'model.pth.tar',map_location=torch.device('cpu'))
new_model.load_state_dict(checkpoint['model'])

_, _, y, y_pred = best_test(new_model,test_loader_2,criterion1,optim,device,1)

_, _, y, y_pred = best_test(new_model,test_loader_cbis,criterion1,optim,device,1)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y, y_pred, average= 'weighted')

In [ ]:
# prob = torch.tensor(y_prob)
# y_true = torch.tensor(y_true)

In [ ]:
# from sklearn import metrics

# fpr, tpr, thresholds = metrics.roc_curve(y_true.numpy(), prob.numpy())

In [ ]:
# roc_auc = metrics.auc(fpr, tpr)
# print(roc_auc)

In [ ]:
# display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='example estimator')
# display.plot()

In [ ]:
_, _,y,y_pred = best_test(new_model,valid_loader,criterion1,optim,device,1)

In [ ]:
_, _,y,y_pred = best_test(new_model,train_loader,criterion1,optim,device,1)

In [ ]:
with open('./storage/' + DFNAME + '.pkl', 'rb') as f:
    data = pickle.load(f) 

In [ ]:
plt.plot(data['cam_loss'], label = 'CAM loss')
# plt.plot(data['valid_loss'], label = 'Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('CAM Loss vs Epoch Curve (CAAM)')
plt.legend()
plt.show()

### Visualization

In [ ]:
plt.plot(data['train_loss'], label = 'Train loss')
plt.plot(data['valid_loss'], label = 'Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch Curve')
plt.legend()
plt.show()

In [ ]:
plt.plot(data['train_acc'], label = 'Train Accuracy')
plt.plot(data['valid_acc'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epoch Curve')
plt.legend()
plt.show()

### TTA 

In [ ]:
test_benign_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/0_benign/")
test_malignant_img_names = os.listdir("/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/1_malignant/")

In [ ]:
all_imgs_names = test_benign_img_names+test_malignant_img_names

In [ ]:
labels = []

for i in range(len(test_benign_img_names)):
    labels.append(0)
    
for i in range(len(test_malignant_img_names)):
    labels.append(1)

In [ ]:
len(labels)

In [ ]:
img_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/combined_test_imgs/'

img_filenames = os.listdir(img_dir)

In [ ]:
def pred_score_func(ori_img, model, transforms):

    img = transforms1(copy.deepcopy(ori_img))
    img = img.to(device)

    outputs,sl_map,ac_map = model(img.unsqueeze(0))

    prob = F.softmax(outputs, dim=1)
    top_p, top_class = prob.topk(1, dim = 1)
    
    return top_p, top_class, outputs

In [ ]:
transforms1 = v2.Compose([
    v2.Resize(size),
    v2.RandomHorizontalFlip(0.5),
    # v2.RandomSolarize(threshold = 0.1,p=1),
    v2.ToTensor(),
    ])

transforms2 = v2.Compose([
    v2.Resize(size),
    v2.RandomRotation(45),
    # v2.RandomPosterize(bits=2,p=0.9),
    v2.ToTensor(),
    ])

transforms3 = v2.Compose([
    v2.Resize(size),
    v2.RandomZoomOut(),
    v2.ToTensor(),
    ])

transforms4 = v2.Compose([
    v2.Resize(size),
    v2.CenterCrop(size=(100, 100)),
    v2.ToTensor(),
    ])

transforms5 = v2.Compose([
    v2.Resize(size),
    v2.RandomInvert(0.5),
    v2.ToTensor(),
    ])

test_transform = v2.Compose([
    v2.Resize(size),
    v2.ToTensor(),
    ])

In [ ]:
from statistics import mean
# i = 42
tta_pred, ori_label, ori_pred = [],[],[]
for i in range(len(img_filenames)):
    pred_per_img, score_per_img = [],[]

    img = Image.open(img_dir + img_filenames[i])
    ori_img = img.convert("RGB") 
    idx = all_imgs_names.index(img_filenames[i])

    ori_label.append(labels[idx])

    top_p, top_class, _ = pred_score_func(ori_img, new_model, transforms1)
    pred_per_img.append(top_class.item())
    score_per_img.append(top_p.item())


    top_p, top_class, _ = pred_score_func(ori_img, new_model, transforms2)
    pred_per_img.append(top_class.item())
    score_per_img.append(top_p.item())


    top_p, top_class, _ = pred_score_func(ori_img, new_model, transforms3)
    pred_per_img.append(top_class.item())
    score_per_img.append(top_p.item())


    top_p, top_class, _ = pred_score_func(ori_img, new_model, transforms4)
    pred_per_img.append(top_class.item())
    score_per_img.append(top_p.item())


    top_p, top_class, _ = pred_score_func(ori_img, new_model, test_transform)
    pred_per_img.append(top_class.item())
    score_per_img.append(top_p.item())
    
    top_p, top_class, outputs = pred_score_func(ori_img, new_model, test_transform)
    _,predicted = torch.max(outputs.data,1)
    
    
    tta_pred.append((sum(pred_per_img)))
    ori_pred.append(predicted.item())

In [ ]:
for i in range(len(tta_pred)):
    print(tta_pred[i], ori_label[i], ori_pred[i])